# Night Pipeline Test

밤 파이프라인 전체 흐름을 테스트합니다.

```
Assets 로드
  ↓
WorldState 조회
  ↓
NightController.process() — NPC 자율행동 (turn 증가 없음)
  - Phase 1: Reflection
  - Phase 2: Planning
  - Phase 3: Group Dialogue (3명 함께)
  - Phase 4: Impact Analysis
  ↓
Delta 적용
  ↓
EndingChecker.check() — 엔딩 체크
  ↓
NarrativeLayer.render_night() 또는 render_ending()
```

## 0. Setup - 프로젝트 루트 탐색, 환경변수 로드

In [ ]:
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")

print(f"Project root: {ROOT}")

## 1. 시나리오 로드 + WorldState 준비

In [ ]:
from app.loader import ScenarioLoader
from app.schemas import WorldState, NPCState
from app.agents.memory import MemoryEntry, add_memory, MEMORY_OBSERVATION

loader = ScenarioLoader(ROOT / "scenarios")
scenario_id = "coraline"  # 코렐라인 시나리오 사용
assets = loader.load(scenario_id)

print(f"시나리오: {assets.scenario.get('title')}")
print(f"NPC: {assets.get_all_npc_ids()}")
print(f"턴 제한: {assets.get_turn_limit()}")

# 테스트용 WorldState - 낮 동안 플레이어가 칼을 만진 상황
world = WorldState(
    turn=3,
    npcs={
        "button_mother": NPCState(npc_id="button_mother", trust=4, fear=0, suspicion=5),
        "button_father": NPCState(npc_id="button_father", trust=3, fear=0, suspicion=6),
        "button_daughter": NPCState(npc_id="button_daughter", trust=5, fear=0, suspicion=4),
    },
    inventory=[],
    vars={
        "humanity": 7,
        "total_suspicion": 6,  # 의심이 쌓인 상태
        "knife_touched": 1,  # 칼을 만졌음!
    },
)

# 낮 동안의 관찰 기억 추가
day_observations = {
    "button_mother": [
        ("새 아이가 부엌에서 칼을 집어들었다. 무슨 생각을 하는 거지?", 9.0),
        ("칼을 들고 문 쪽을 봤다. 탈출하려는 건가?", 8.5),
    ],
    "button_father": [
        ("새 아이가 규칙을 어겼다. 도구를 만지면 안 된다고 했다.", 9.0),
        ("눈빛이 두려움으로 가득했다. 아직 이곳에 적응 못 했어.", 8.0),
    ],
    "button_daughter": [
        ("언니/오빠가 칼을 숨겼어! 무서워...", 9.5),
        ("엄마 아빠한테 말해야 하나? 위험할 것 같아.", 8.5),
    ],
}

for npc_id, obs_list in day_observations.items():
    for desc, imp in obs_list:
        entry = MemoryEntry.create(
            npc_id=npc_id,
            description=desc,
            importance_score=imp,
            current_turn=world.turn - 1,
            memory_type=MEMORY_OBSERVATION,
        )
        add_memory(world.npcs[npc_id].extras, entry)

# 성찰 트리거를 위한 accumulated_importance 설정
for npc_id in world.npcs:
    world.npcs[npc_id].extras["accumulated_importance"] = 30.0

print(f"\n초기 상태:")
print(f"  턴: {world.turn}")
print(f"  인간성: {world.vars['humanity']}")
print(f"  총 의심도: {world.vars['total_suspicion']}")
print(f"  칼 터치: {'예' if world.vars['knife_touched'] else '아니오'}")
print(f"\n[NPC 초기 메모리]")
for npc_id, npc in world.npcs.items():
    stream = npc.extras.get("memory_stream", [])
    acc = npc.extras.get("accumulated_importance", 0)
    print(f"  {npc_id}: memories={len(stream)}, acc_importance={acc:.1f}")

## 2. NightController - 밤 페이즈 실행

In [ ]:
from app.night_controller import get_night_controller

night_controller = get_night_controller()

print("=== NightController 실행 ===")
print("밤이 되었습니다. NPC들이 자율적으로 행동합니다.\n")

night_result = night_controller.process(world, assets)

print(f"\n[Night 결과]")
print(f"  그룹 대화: {len(night_result.night_conversation)}개 발화")
print(f"  스탯 변화: {night_result.night_delta.get('npc_stats', {})}")
print(f"  변수 변화: {night_result.night_delta.get('vars', {})}")
print(f"  턴 증가: {night_result.night_delta.get('turn_increment', 0)}")

## 3. 그룹 대화 내용 확인

In [ ]:
print("=== 밤의 가족 회의 ===")
print("단추 인형 가족이 모여 새 아이에 대해 논의합니다.\n")
print("-" * 60)

for i, utt in enumerate(night_result.night_conversation, 1):
    speaker = utt.get("speaker", "")
    text = utt.get("text", "")
    print(f"[{i}] {speaker}: {text}")

print("-" * 60)

## 4. Delta 적용 - WorldState 업데이트

In [ ]:
from app.state import get_world_state_manager

wsm = get_world_state_manager()
user_id = "test_user"

print("=== Delta 적용 ===")
print(f"적용 전 턴: {world.turn}")

# Delta 적용
world_after = wsm.apply_delta(user_id, scenario_id, night_result.night_delta, assets)

print(f"적용 후 턴: {world_after.turn} (밤은 턴 증가 없음)")
print(f"\n변경된 상태:")
print(f"  인간성: {world.vars.get('humanity')} → {world_after.vars.get('humanity')}")
print(f"  총 의심도: {world.vars.get('total_suspicion')} → {world_after.vars.get('total_suspicion')}")

for npc_id in world.npcs:
    before = world.npcs[npc_id]
    after = world_after.npcs[npc_id]
    npc_data = assets.get_npc_by_id(npc_id) or {}
    npc_name = npc_data.get("name", npc_id)
    print(f"\n  [{npc_name}]")
    print(f"    trust: {before.trust} → {after.trust}")
    print(f"    suspicion: {before.suspicion} → {after.suspicion}")

## 5. EndingChecker - 엔딩 조건 체크

In [ ]:
from app.ending_checker import check_ending

print("=== EndingChecker ===")
ending_result = check_ending(world_after, assets)

if ending_result.reached:
    print(f"\n🎬 엔딩 도달!")
    print(f"  엔딩 ID: {ending_result.ending.ending_id}")
    print(f"  엔딩 이름: {ending_result.ending.name}")
    print(f"  epilogue_prompt: {ending_result.ending.epilogue_prompt[:100]}...")
    
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
else:
    print("엔딩 미도달 - 게임 계속")
    ending_info = None

## 6. NarrativeLayer - 밤 나레이션 생성

In [ ]:
from app.narrative import get_narrative_layer

narrative = get_narrative_layer()

print("=== NarrativeLayer ===")

if ending_info:
    # 엔딩 나레이션
    print("\n[엔딩 나레이션 생성]")
    dialogue = narrative.render_ending(ending_info, world_after, assets)
else:
    # 밤 나레이션 (그룹 대화를 소설 형식으로)
    print("\n[밤 나레이션 생성]")
    dialogue = narrative.render_night(
        world_after,
        assets,
        night_result.night_conversation,
    )

print("\n" + "=" * 60)
print(dialogue)
print("=" * 60)

## 7. NPC 메모리 및 계획 확인

In [ ]:
print("=== NPC 내부 상태 (성찰 & 계획) ===")

for npc_id, npc_state in world_after.npcs.items():
    npc_data = assets.get_npc_by_id(npc_id) or {}
    npc_name = npc_data.get("name", npc_id)
    
    stream = npc_state.extras.get("memory_stream", [])
    plan = npc_state.extras.get("current_plan", {}).get("plan_text", "없음")
    acc_importance = npc_state.extras.get("accumulated_importance", 0)
    
    print(f"\n[{npc_name}]")
    print(f"  메모리 수: {len(stream)}개")
    print(f"  누적 중요도: {acc_importance:.1f}")
    print(f"  현재 계획: {plan[:80]}..." if len(plan) > 80 else f"  현재 계획: {plan}")
    
    # 최근 성찰 내용 (reflection 타입만)
    reflections = [m for m in stream if m.get("memory_type") == "reflection"]
    if reflections:
        recent = reflections[-1]
        print(f"  최근 성찰: {recent.get('description', '')[:60]}...")

## 8. 전체 파이프라인 결과 요약

In [ ]:
print("=" * 60)
print("Night Pipeline 테스트 완료")
print("=" * 60)
print(f"\n최종 상태:")
print(f"  턴: {world.turn} → {world_after.turn} (밤은 증가 없음)")
print(f"  인간성: {world.vars.get('humanity')} → {world_after.vars.get('humanity')}")
print(f"  총 의심도: {world.vars.get('total_suspicion')} → {world_after.vars.get('total_suspicion')}")
print(f"  엔딩 도달: {'예 (' + ending_info['name'] + ')' if ending_info else '아니오'}")
print(f"\n그룹 대화: {len(night_result.night_conversation)}개 발화")
print(f"생성된 나레이션: {len(dialogue)}자")
print(f"\nDebug Info:")
print(f"  - NightController: {len(night_result.night_conversation)}개 발화")

# NPC별 메모리 변화
print(f"\n  [NPC 메모리 변화]")
for npc_id in world.npcs:
    before_count = len(world.npcs[npc_id].extras.get("memory_stream", []))
    after_count = len(world_after.npcs[npc_id].extras.get("memory_stream", []))
    print(f"    {npc_id}: {before_count} → {after_count} (+{after_count - before_count})")

print(f"  - EndingChecker: {'reached' if ending_result.reached else 'not reached'}")
print(f"  - NarrativeLayer: {'ending' if ending_info else 'night'} mode")